In [1]:
%pylab widget

Populating the interactive namespace from numpy and matplotlib


In [2]:
#!/bin/python3

from physdata.xray import *  # pip install physdata
import numpy as np

element_density = []

for element in fetch_elements():
    
    element_density.append(element.density)

In [4]:
import re

names = []
weights = []
elements = []
density = []

element_number = []
# element_density = []

with open('../data/geant4_materials.txt','r') as f:
    
    lines = f.readlines()
        
    for ii in range(len(lines)):
        
        line = lines[ii]
        
        if ' AddMaterial' in line:
            
            seps = line.split('"')
            
            if len(seps) > 1:
                
                if len(seps[1]) > 5:
                    
                    jj = ii + 1
                    
                    if ' AddElementByAtom' in lines[jj]:
                        continue
                    
                    weight = []
                    element = []
                    
                    while ' AddElementByWeightFraction' in lines[jj]:
                        
                        numbers = re.split('[\(\)]',lines[jj])[1].split(',')
                        
                        element.append(int(numbers[0]))
                        weight.append(float(numbers[1]))
                        
                        jj += 1
                    
                    elements.append(element)
                    weights.append(weight)    
                    names.append(seps[1])
                    
                    density.append(float(seps[2].split(',')[1]))
                    
                else:
                    element_number.append(int(seps[2].split(',')[2]))
#                     element_density.append(float(seps[2].split(',')[1]))

In [5]:
# names_mod = [name if name[ii][1:2] == 'G4' for name in names]
names_mod = [(name) for name in names if name[:2]!="G4"]
inds_mod = [(ii) for ii,name in enumerate(names) if name[:2]!="G4"]

In [6]:
_elements = ['Hydrogen', 'Helium', 'Lithium', 'Beryllium', 'Boron', 'Carbon', 'Nitrogen', 'Oxygen',
             'Fluorine', 'Neon', 'Sodium', 'Magnesium', 'Aluminum', 'Silicon', 'Phosphorus', 'Sulfur', 'Chlorine',
             'Argon', 'Potassium', 'Calcium', 'Scandium', 'Titanium', 'Vanadium', 'Chromium', 'Manganese', 'Iron',
             'Cobalt', 'Nickel', 'Copper', 'Zinc', 'Gallium', 'Germanium', 'Arsenic', 'Selenium', 'Bromine', 'Krypton',
             'Rubidium', 'Strontium', 'Yttrium', 'Zirconium', 'Niobium', 'Molybdenum', 'Technetium', 'Ruthenium',
             'Rhodium', 'Palladium', 'Silver', 'Cadmium', 'Indium', 'Tin', 'Antimony', 'Tellurium', 'Iodine', 'Xenon',
             'Cesium', 'Barium', 'Lanthanum', 'Cerium', 'Praseodymium', 'Neodymium', 'Promethium', 'Samarium',
             'Europium', 'Gadolinium', 'Terbium', 'Dysprosium', 'Holmium', 'Erbium', 'Thulium', 'Ytterbium', 'Lutetium',
             'Hafnium', 'Tantalum', 'Tungsten', 'Rhenium', 'Osmium', 'Iridium', 'Platinum', 'Gold', 'Mercury',
             'Thallium', 'Lead', 'Bismuth', 'Polonium', 'Astatine', 'Radon', 'Francium', 'Radium', 'Actinium',
             'Thorium', 'Protactinium', 'Uranium']

In [27]:
colours = ["TransparentBlack","blue","orange","yellow","skyblue","white","grey","green","red"]

In [29]:
with open('../../scintillators/XCAT/custom_materials_topas.txt','w') as f:
    for ind in range(len(names)):
        
        if '-' in names[ind]:
            continue
        f.writelines('sv:Ma/' + 'XCAT_' + names[ind] + '/Components = ' + str(len(elements[ind]))
               + ' "' + '" "'.join([_elements[element-1] for element in elements[ind]]) + '"' + '\n')
        f.writelines('uv:Ma/' + 'XCAT_' + names[ind] + '/Fractions = ' + str(len(elements[ind]))
               + ' ' + ' '.join([str(weight) for weight in weights[ind]])+ '\n')
        f.writelines('d:Ma/' + 'XCAT_' + names[ind] + '/Density = ' + str(density[ind]) + ' g/cm3 \n')
        f.writelines('s:Ma/' + 'XCAT_' + names[ind] + '/DefaultColor = "' + colours[ind%len(colours)] + '"\n' )
        f.writelines('\n')

In [7]:
'" "'.join([str(element) for element in elements[ind]])

'8" "6" "1" "7" "20" "15" "19" "16" "11" "17" "12" "26'

In [72]:
ind

187

In [4]:
H = np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/1.csv',delimiter=',')

import fastcat.fastcat as fc

for kk, name in enumerate(names):
    
    attenuations = []
    energies = []
    lens = []
    
    for weight, element in zip(weights[kk],elements[kk]):
        
        attenuation = np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/'+str(element)+'.csv',delimiter=',')[1]
        attenuation = attenuation/element_density[element-1]*weight
        
#         print(element_density[element-1])
        
        attenuations.append(attenuation)
        energies.append(np.loadtxt('/home/xcite/fastcat-0.0.1/fastcat/data/mu/'+str(element)+'.csv',delimiter=',')[0])
        lens.append(len(attenuation))
        
    if max(lens) == 36:
        
        attenuation_all = np.zeros([2,len(H[0])])
        attenuation_all[0] = H[0]
        
        for ii in range(len(attenuations)):
            
            attenuation_all[1] += attenuations[ii]
        
    else:
        
        energies_larger = np.unique(np.hstack(energies))
        
        attenuation_all = np.zeros([2,len(energies_larger)])
        attenuation_all[0] = energies_larger
        
        for ii in range(len(attenuations)):
            
            f = fc.log_interp_1d(energies[ii],attenuations[ii])
            
            attenuation_all[1] += f(energies_larger)
            
        
    attenuation_all[1] *= density[kk]
    np.savetxt('elements/'+name+".csv",attenuation_all, fmt="%.8G",delimiter=",")
    
#     if name == 'G4_TEFLON':
#         break

In [7]:
!mv elements/* ../fastcat/data/mu/

In [197]:
np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'9'+'.csv',delimiter=',')[1]/element_density[8]

array([5.649e+03, 1.979e+03, 9.047e+02, 2.888e+02, 1.256e+02, 6.514e+01,
       3.789e+01, 1.602e+01, 8.205e+00, 2.492e+00, 1.133e+00, 4.487e-01,
       2.828e-01, 2.214e-01, 1.920e-01, 1.639e-01, 1.496e-01, 1.298e-01,
       1.176e-01, 1.015e-01, 9.073e-02, 8.274e-02, 7.649e-02, 6.717e-02,
       6.037e-02, 5.399e-02, 4.915e-02, 4.228e-02, 3.422e-02, 2.960e-02,
       2.663e-02, 2.457e-02, 2.195e-02, 2.039e-02, 1.846e-02, 1.769e-02])

In [202]:
(np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'9'+'.csv',delimiter=',')[1]/element_density[8]*.76 +
 np.loadtxt('/home/xcite/xpecgen/xpecgen/data/mu/'+'6'+'.csv',delimiter=',')[1]/element_density[5]*.24)*density[kk]

array([1.0612536e+04, 3.6785936e+03, 1.6724312e+03, 5.3056784e+02,
       2.2995104e+02, 1.1900944e+02, 6.9133680e+01, 2.9201568e+01,
       1.4971704e+01, 4.5927728e+00, 2.1277520e+00, 8.8550000e-01,
       5.8245440e-01, 4.6896960e-01, 4.1358240e-01, 3.5904880e-01,
       3.3007040e-01, 2.8814720e-01, 2.6151840e-01, 2.2599280e-01,
       2.0210344e-01, 1.8435648e-01, 1.7043752e-01, 1.4966952e-01,
       1.3452472e-01, 1.2031448e-01, 1.0952392e-01, 9.4145920e-02,
       7.6023200e-02, 6.5579360e-02, 5.8823600e-02, 5.4117360e-02,
       4.8073520e-02, 4.4435600e-02, 3.9830560e-02, 3.7893680e-02])

In [205]:
attenuation_all[1]

array([1.06111519e+04, 3.67807876e+03, 1.67218879e+03, 5.30487936e+02,
       2.29915684e+02, 1.18990912e+02, 6.91228340e+01, 2.91969606e+01,
       1.49693560e+01, 4.59209446e+00, 2.12747380e+00, 8.85422499e-01,
       5.82424124e-01, 4.68955791e-01, 4.13575677e-01, 3.59047632e-01,
       3.30071125e-01, 2.88149173e-01, 2.61520534e-01, 2.25994853e-01,
       2.02105344e-01, 1.84358255e-01, 1.70439167e-01, 1.49670965e-01,
       1.34526024e-01, 1.20315652e-01, 1.09524983e-01, 9.41467816e-02,
       7.60237636e-02, 6.55797103e-02, 5.88237812e-02, 5.41174083e-02,
       4.80733549e-02, 4.44352779e-02, 3.98299642e-02, 3.78928990e-02])

In [188]:
weights[kk][0]

0.240183

In [75]:
physdata.xray.ElementData()

In [67]:
H[0].shape

(36,)

In [69]:
elements[kk]

[1, 6, 7, 8, 9, 20]